# Factorial Kriging Analysis

<!-- SUMMARY: Filtering images using Factorial Kriging Analysis. Example on a multivariate acquisition  -->

<!-- CATEGORY: Courses -->

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import os

gdoc.setNoScroll()
gl.OptCst.define(gl.ECst.NTROW, -1)
gl.OptCst.define(gl.ECst.NTCOL, -1)
gl.OptCst.define(gl.ECst.NTCAR, 15)

figsize=(8,8)

The Grid containing the information is downloaded from the distribution.

The loaded file (called **grid **) contains 3 variables:

- **P** (phosphorus) which is the *variable of interest*
- **Cr** (chromium) is an auxiliary variable
- **Ni** (nickel) another auxiliary variable


In [ ]:
fileNF = gdoc.loadData("FKA", "Image.ascii")
grid = gl.DbGrid.createFromNF(fileNF)
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim)

In [ ]:
dbfmt = gl.DbStringFormat()
dbfmt.setFlags(flag_resume=False,flag_vars=False,flag_stats=True, names="P")
grid.display(dbfmt)

Note that some pixels are not informed for variable **P**.

Statistics on auxiliary variables

In [ ]:
dbfmt.setFlags(flag_vars=False, flag_resume=True, flag_stats=True, names=["Cr", "Ni"])
grid.display(dbfmt)

Correlation between variables

In [ ]:
res = gp.correlation(grid, namex="Cr", namey="P", bins=100, cmin=1)

In [ ]:
res = gp.correlation(grid, namex="Ni", namey="P", bins=100, cmin=1)

In [ ]:
res = gp.correlation(grid, namex="Ni", namey="Cr", bins=100, cmin=1)

Using inverse square distance for completing the variable **P**

In [ ]:
grid.setLocator("P", gl.ELoc.Z)
err = gl.DbHelper.dbgrid_filling(grid,0,13432,1)

We concentrate on the variable of interest **P** completed (**Fill.P**) in the next operations

In [ ]:
fig, ax = gp.init(figsize=figsize, flagEqual=True)
ax.raster(grid,"Fill.P")
gp.close()

Variogram Calculation along Grid main axes

In [ ]:
varnames = ["Fill.P"]
varioparam = gl.VarioParam.createMultipleFromGrid(grid, nlag=100)
varioP = gl.Vario(varioparam)
err = varioP.compute(grid)
modelP = gl.Model()
types = [gl.ECov.NUGGET, gl.ECov.SPHERICAL]
err = modelP.fit(varioP, types=types, optvar=gl.Option_VarioFit(True, False))
modelP.setDriftIRF(0,0)

modelP.setCovFiltered(0, True)
means = gl.dbStatisticsMono(grid,varnames,[gl.EStatOption.MEAN]).getValues()
modelP.setMeans(means)

modelP

In [ ]:
res = gp.varmod(varioP, modelP)

We must define the **Neighborhood**

In [ ]:
neigh = gl.NeighImage([5,5])

The **image** neighborhood is based on $(2*10+1)^2=441$ pixels (centered on the target pixel).

During the estimation, only the contribution of second and third basic structures are kept (Nugget Effect is filtered out): **Factorial Kriging Analysis**.

In [ ]:
err  = gl.krimage(grid,modelP,neigh,flagFFT=True, verbose=True,namconv=gl.NamingConvention("Mono"))

In [ ]:
fig, ax = gp.init(1,2, figsize=figsize, flagEqual=True)
ax[0].raster(grid,"Fill.P")
ax[1].raster(grid,"Mono*.P")
fig.decoration(title="Filtering P (monovariate)")

Correlation for **P** variable between Initial image (completed) and its Filtered version (monovariate FKA)

In [ ]:
gp.correlation(grid, namex="Fill.P", namey="Mono.Fill.P", bins=100, cmin=1)
gp.decoration(xlabel="P Filled",ylabel="P Filtered (Mono)")

## Multivariate approach

Having a look at the two covariables

In [ ]:
fig, ax = gp.init(1,2, figsize=figsize, flagEqual=True)
ax[0].raster(grid,"Cr")
ax[1].raster(grid,"Ni")
fig.decoration(title="Covariables")

In [ ]:
varnames = ["Fill.P", "Ni", "Cr"]
grid.setLocators(varnames, gl.ELoc.Z)

varioM = gl.Vario(varioparam)
err = varioM.compute(grid)
modelM = gl.Model()
err = modelM.fit(varioM, types=types, optvar=gl.Option_VarioFit(True, False))
modelM.setDriftIRF(0,0)

modelM.setCovFiltered(0, True)
means = gl.dbStatisticsMono(grid,varnames,[gl.EStatOption.MEAN]).getValues()
modelM.setMeans(means)
modelM

In [ ]:
res = gp.varmod(varioM, modelM)

Multivariable Factorial Kriging Analysis

In [ ]:
err  = gl.krimage(grid,modelM,neigh,flagFFT=True,verbose=True,namconv=gl.NamingConvention("Multi"))

Note that, using the same **neigh** as in monovariate, the dimension of the Kriging System is now $3 * 441 = 1323$

In [ ]:
fig, ax = gp.init(1,2, figsize=figsize, flagEqual=True)
ax[0].raster(grid,"Mono*.P")
ax[1].raster(grid,"Multi*.P")
fig.decoration(title="P filtered")

Correlation for **P** variable between Initial image and its Filtered version (multivariate FKA)

In [ ]:
gp.correlation(grid, namex="Fill.P", namey="Multi.Fill.P", bins=100, cmin=1)
gp.decoration(xlabel="P Filled",ylabel="P Filtered (Multi)")

Correlation for **P** filtered variable between the Monovariate and the Multivariate case

In [ ]:
gp.correlation(grid, namex="Mono.Fill.P", namey="Multi.Fill.P", bins=100, cmin=1)
gp.decoration(xlabel="P Filtered (Mono)",ylabel="P Filtered (Multi)")